In [ ]:
from firebase_admin import firestore, initialize_app, credentials
from gcp_utils import constants

cred = credentials.Certificate('/workspaces/github/heartfelt-0-firebase-adminsdk-unf3t-f7f484fd88.json')
initialize_app(cred)
database = firestore.client()
col = database.collection(u'bpm_data')

frame = constants.BPM_FRAME
frame['target'] = 'bpm_data'

In [ ]:
col.document('test_user').collection('frames').add(frame)

In [ ]:
# Cam UID
col.document('wx1jF08b3DTPijtQcwGiEwpEFai2').collection('frames').add(frame)

In [ ]:
# Aslani UID
col.document('v2iHQmPIVfVW0IuhfZ1yCIegsB52').collection('frames').add(frame)